# Sentiment Analysis Model

## Training the Data

In [1]:
import pandas as pd
import numpy as np

#NLTK
import nltk
from nltk import word_tokenize, WordPunctTokenizer, regexp_tokenize
from nltk.corpus import stopwords

#Keras
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

#Sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB

#Plotting 
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

## TASS Workshop

Source:

TASS-2017: Workshop on Semantic Analysis at SEPLN (http://www.sepln.org/workshops/tass/2017/)

TASS Workshop on Semantic Analysis has been held since 2012, under the umbrella of the International Conference of the Spanish Society for Natural Language Processing (SEPLN). TASS was the first shared task on sentiment analysis in Twitter in Spanish. The initial aim of TASS was the furtherance of research on sentiment analysis in Spanish with a special interest on the language used in Twitter.

In [2]:
import xml.etree.ElementTree as ET
tree = ET.parse('Resources/general-train-tagged-3l.xml')
root = tree.getroot()

In [3]:
train_data = pd.DataFrame({'tweetText':[],'polarity_value':[]})
row=0
for tweet in root:
    tweetText = tweet.find('content').text
    lang = tweet.find('lang').text
    polarity_value = tweet.find('sentiments').find('polarity').find('value').text

    if lang == 'es':
        train_data.loc[row] = [tweetText,polarity_value]
        row+=1

train_data

,tweetText,polarity_value
0,"Salgo de #VeoTV , que día más largoooooo...",NONE
1,@PauladeLasHeras No te libraras de ayudar me/n...,NEU
2,@marodriguezb Gracias MAR,P
3,"Off pensando en el regalito Sinde, la que se v...",N
4,Conozco a alguien q es adicto al drama! Ja ja ...,P
...,...,...
7213,Muy indignante si ...nadie repara en ello hoy ...,N
7214,Más pobres por discriminar a la mujer http://t...,N
7215,Crean un banco de productos para mujeres con c...,P
7216,Sobre la sorprendente huída hoy en el Senado d...,N


In [4]:
train_data = train_data.set_index("polarity_value")
train_data

,tweetText
polarity_value,
NONE,"Salgo de #VeoTV , que día más largoooooo..."
NEU,@PauladeLasHeras No te libraras de ayudar me/n...
P,@marodriguezb Gracias MAR
N,"Off pensando en el regalito Sinde, la que se v..."
P,Conozco a alguien q es adicto al drama! Ja ja ...
...,...
N,Muy indignante si ...nadie repara en ello hoy ...
N,Más pobres por discriminar a la mujer http://t...
P,Crean un banco de productos para mujeres con c...


In [5]:
train_data = train_data.drop("NONE", axis=0)
train_data

,tweetText
polarity_value,
NEU,@PauladeLasHeras No te libraras de ayudar me/n...
P,@marodriguezb Gracias MAR
N,"Off pensando en el regalito Sinde, la que se v..."
P,Conozco a alguien q es adicto al drama! Ja ja ...
P,Toca @crackoviadeTV3 . Grabación dl especial N...
...,...
N,Muy indignante si ...nadie repara en ello hoy ...
N,Más pobres por discriminar a la mujer http://t...
P,Crean un banco de productos para mujeres con c...


In [6]:
train_data = train_data.reset_index()
train_data

,polarity_value,tweetText
0,NEU,@PauladeLasHeras No te libraras de ayudar me/n...
1,P,@marodriguezb Gracias MAR
2,N,"Off pensando en el regalito Sinde, la que se v..."
3,P,Conozco a alguien q es adicto al drama! Ja ja ...
4,P,Toca @crackoviadeTV3 . Grabación dl especial N...
...,...,...
5731,N,Muy indignante si ...nadie repara en ello hoy ...
5732,N,Más pobres por discriminar a la mujer http://t...
5733,P,Crean un banco de productos para mujeres con c...
5734,N,Sobre la sorprendente huída hoy en el Senado d...


In [7]:
train_data['polarity_value'] = train_data['polarity_value'].replace(['NEU'],0)
train_data['polarity_value'] = train_data['polarity_value'].replace(['P'],1)
train_data['polarity_value'] = train_data['polarity_value'].replace(['N'],-1)
train_data

,polarity_value,tweetText
0,0,@PauladeLasHeras No te libraras de ayudar me/n...
1,1,@marodriguezb Gracias MAR
2,-1,"Off pensando en el regalito Sinde, la que se v..."
3,1,Conozco a alguien q es adicto al drama! Ja ja ...
4,1,Toca @crackoviadeTV3 . Grabación dl especial N...
...,...,...
5731,-1,Muy indignante si ...nadie repara en ello hoy ...
5732,-1,Más pobres por discriminar a la mujer http://t...
5733,1,Crean un banco de productos para mujeres con c...
5734,-1,Sobre la sorprendente huída hoy en el Senado d...


In [8]:
train_data = train_data[['tweetText','polarity_value']]
train_data

,tweetText,polarity_value
0,@PauladeLasHeras No te libraras de ayudar me/n...,0
1,@marodriguezb Gracias MAR,1
2,"Off pensando en el regalito Sinde, la que se v...",-1
3,Conozco a alguien q es adicto al drama! Ja ja ...,1
4,Toca @crackoviadeTV3 . Grabación dl especial N...,1
...,...,...
5731,Muy indignante si ...nadie repara en ello hoy ...,-1
5732,Más pobres por discriminar a la mujer http://t...,-1
5733,Crean un banco de productos para mujeres con c...,1
5734,Sobre la sorprendente huída hoy en el Senado d...,-1


In [9]:
train_data.polarity_value.value_counts()

 1    2884
-1    2182
 0     670
Name: polarity_value, dtype: int64

In [10]:
train_data['processed_tweet'] = train_data.tweetText

<ipython-input-10-4fab69b2716d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['processed_tweet'] = train_data.tweetText


In [11]:
train_data

,tweetText,polarity_value,processed_tweet
0,@PauladeLasHeras No te libraras de ayudar me/n...,0,@PauladeLasHeras No te libraras de ayudar me/n...
1,@marodriguezb Gracias MAR,1,@marodriguezb Gracias MAR
2,"Off pensando en el regalito Sinde, la que se v...",-1,"Off pensando en el regalito Sinde, la que se v..."
3,Conozco a alguien q es adicto al drama! Ja ja ...,1,Conozco a alguien q es adicto al drama! Ja ja ...
4,Toca @crackoviadeTV3 . Grabación dl especial N...,1,Toca @crackoviadeTV3 . Grabación dl especial N...
...,...,...,...
5731,Muy indignante si ...nadie repara en ello hoy ...,-1,Muy indignante si ...nadie repara en ello hoy ...
5732,Más pobres por discriminar a la mujer http://t...,-1,Más pobres por discriminar a la mujer http://t...
5733,Crean un banco de productos para mujeres con c...,1,Crean un banco de productos para mujeres con c...
5734,Sobre la sorprendente huída hoy en el Senado d...,-1,Sobre la sorprendente huída hoy en el Senado d...


## AMLO Tweets

5 days before/after the video scandal from Pio Lopez Obrador

In [12]:
amlo_tweets = pd.read_csv('Resources/Tweets.csv')
amlo_tweets

,Content,Date,User
0,"Sí, también misóginos y arribistas.",30 ago.,@laishawilkins
1,No puedo entender como “hombres” que se presum...,30 ago.,@serabenh
2,Comunista de ultra derecha,29 ago.,@MACNASRUDIN
3,Gran presidente apoyo total a nuestro presiden...,30 ago.,@ArmandoZermeno2
4,"Con la panzota que se carga ??? No mame , no e...",31 ago.,@gerryboy10
...,...,...,...
1055,@lopezobrador_ Y vemos en tv cualquier cantida...,1 ago.,@YOSOYIZQUIERDA2
1056,@lopezobrador_ La trata de personas vemos un ...,1 ago.,@YOSOYIZQUIERDA2
1057,@lopezobrador_ debe estar muy orgulloso por ha...,1 ago.,@TaCabronNo1
1058,@lopezobrador_ anunciará el 15 de Septiembre q...,1 ago.,@ElGuyTrejo


In [13]:
amlo_tweets=amlo_tweets.rename(columns={'Content':'tweetText'})
amlo_tweets

,tweetText,Date,User
0,"Sí, también misóginos y arribistas.",30 ago.,@laishawilkins
1,No puedo entender como “hombres” que se presum...,30 ago.,@serabenh
2,Comunista de ultra derecha,29 ago.,@MACNASRUDIN
3,Gran presidente apoyo total a nuestro presiden...,30 ago.,@ArmandoZermeno2
4,"Con la panzota que se carga ??? No mame , no e...",31 ago.,@gerryboy10
...,...,...,...
1055,@lopezobrador_ Y vemos en tv cualquier cantida...,1 ago.,@YOSOYIZQUIERDA2
1056,@lopezobrador_ La trata de personas vemos un ...,1 ago.,@YOSOYIZQUIERDA2
1057,@lopezobrador_ debe estar muy orgulloso por ha...,1 ago.,@TaCabronNo1
1058,@lopezobrador_ anunciará el 15 de Septiembre q...,1 ago.,@ElGuyTrejo


In [14]:
amlo_tweets = amlo_tweets[["tweetText"]]
amlo_tweets

,tweetText
0,"Sí, también misóginos y arribistas."
1,No puedo entender como “hombres” que se presum...
2,Comunista de ultra derecha
3,Gran presidente apoyo total a nuestro presiden...
4,"Con la panzota que se carga ??? No mame , no e..."
...,...
1055,@lopezobrador_ Y vemos en tv cualquier cantida...
1056,@lopezobrador_ La trata de personas vemos un ...
1057,@lopezobrador_ debe estar muy orgulloso por ha...
1058,@lopezobrador_ anunciará el 15 de Septiembre q...


In [15]:
amlo_tweets['processed_tweet'] = amlo_tweets.tweetText
amlo_tweets

<ipython-input-15-9d4134a5aeba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amlo_tweets['processed_tweet'] = amlo_tweets.tweetText


,tweetText,processed_tweet
0,"Sí, también misóginos y arribistas.","Sí, también misóginos y arribistas."
1,No puedo entender como “hombres” que se presum...,No puedo entender como “hombres” que se presum...
2,Comunista de ultra derecha,Comunista de ultra derecha
3,Gran presidente apoyo total a nuestro presiden...,Gran presidente apoyo total a nuestro presiden...
4,"Con la panzota que se carga ??? No mame , no e...","Con la panzota que se carga ??? No mame , no e..."
...,...,...
1055,@lopezobrador_ Y vemos en tv cualquier cantida...,@lopezobrador_ Y vemos en tv cualquier cantida...
1056,@lopezobrador_ La trata de personas vemos un ...,@lopezobrador_ La trata de personas vemos un ...
1057,@lopezobrador_ debe estar muy orgulloso por ha...,@lopezobrador_ debe estar muy orgulloso por ha...
1058,@lopezobrador_ anunciará el 15 de Septiembre q...,@lopezobrador_ anunciará el 15 de Septiembre q...


## Data Processing

### Hashtags

In [16]:
import re
hash_regex = re.compile(r"#(\w+)")
hashtags = [] 
def hash_repl(match):
    _ = '__HASH__'+match.group(1).upper()
    hashtags.append(_)
    return _

### URL

In [17]:
url_regex = re.compile(r"(http|https|ftp)://[a-zA-Z0-9\./]+")
def url_repl(match):
    return '__URL__'

### Repetitions

In [18]:
rpt_regex = re.compile(r"(.)\1{1,}", re.IGNORECASE);
def rpt_repl(match):
    return match.group(1)+match.group(1)

In [19]:
# Testing
re.sub(rpt_regex, rpt_repl, "Reppppppeated characters in wordsssss" )

'Reppeated characters in wordss'

### Usernames

In [20]:
user_regex = re.compile(r"@(\w+)")
usr_names = [] # To store the user names so we can exclude them from some parts of the analysis
def user_repl(match):
    _ = '__USER__'+match.group(1).upper()
    usr_names.append(_)
    return _

### Punctuation

In [21]:
# Spliting by word boundaries
word_bound_regex = re.compile(r"\W+")

# Punctuations
punctuations = \
    [   
        ('__PEXCL__',    ['!', '¡', ] )    ,\
        ('__PQUES__',    ['?', '¿', ] )    ,\
        ('__PPROG__',    ['...', '…', ] )  ,\
    ]

#For punctuation replacement
def punctuations_repl(match):
    text = match.group(0)
    repl = []
    for (key, parr) in punctuations :
        for punc in parr :
            if punc in text:
                repl.append(key)
    if(len(repl)>0 ) :
        return ' '+' '.join(repl)+' '
    else :
        return ' '

### Stopwords

In [22]:
#nltk.download('stopwords')
#stop_words=stopwords.words('spanish')

#def stopwords(text):
#    text = [w for w in text if not w in stop_words]
    
#    return(text)

In [23]:
#stop_words

### Further Cleaning & Stemming

In [24]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish', ignore_stopwords = True)

In [25]:
def clean(text):
    text = re.sub('\w+', lambda x:'' if x.group().startswith('__') else x.group(), text)
    
    return(text)

In [26]:
def processAll(text):
    text = re.sub( hash_regex, hash_repl, text )
    text = re.sub( user_regex, user_repl, text)
    text = re.sub( url_regex, url_repl, text )
    
    text = text.replace('\'','')
    
    text = re.sub( word_bound_regex , punctuations_repl, text )
    text = re.sub( rpt_regex, rpt_repl, text )
    
    text = clean(text)
    #text = stopwords(text)
    
    return text

In [27]:
train_data['processed_tweet'] = train_data.tweetText.apply(processAll)
amlo_tweets['processed_tweet'] = amlo_tweets.tweetText.apply(processAll)

<ipython-input-27-88a2a791a33d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['processed_tweet'] = train_data.tweetText.apply(processAll)
<ipython-input-27-88a2a791a33d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amlo_tweets['processed_tweet'] = amlo_tweets.tweetText.apply(processAll)


In [28]:
train_data

,tweetText,polarity_value,processed_tweet
0,@PauladeLasHeras No te libraras de ayudar me/n...,0,No te libraras de ayudar me nos Besos y gracias
1,@marodriguezb Gracias MAR,1,Gracias MAR
2,"Off pensando en el regalito Sinde, la que se v...",-1,Off pensando en el regalito Sinde la que se va...
3,Conozco a alguien q es adicto al drama! Ja ja ...,1,Conozco a alguien q es adicto al drama Ja ja ...
4,Toca @crackoviadeTV3 . Grabación dl especial N...,1,Toca Grabación dl especial Navideño Mari cri...
...,...,...,...
5731,Muy indignante si ...nadie repara en ello hoy ...,-1,Muy indignante si nadie repara en ello hoy aq...
5732,Más pobres por discriminar a la mujer http://t...,-1,Más pobres por discriminar a la mujer
5733,Crean un banco de productos para mujeres con c...,1,Crean un banco de productos para mujeres con c...
5734,Sobre la sorprendente huída hoy en el Senado d...,-1,Sobre la sorprendente huída hoy en el Senado d...


In [29]:
def sb_stem(text):
    text = [word if(word[0:2]=='__') else word.lower() for word in text.split() if ((len(word) >= 3) or (word in ['no','si', 'sí', 'ni']))] #keep the small words (like 'no')
    text = [stemmer.stem(w) if w[0:2]!='__' else w for w in text ]
    
    return(text)

In [30]:
train_data['stemmed_tweet'] = train_data.processed_tweet.apply(sb_stem)
amlo_tweets['stemmed_tweet'] = amlo_tweets.processed_tweet.apply(sb_stem)

<ipython-input-30-e0a594fbf798>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['stemmed_tweet'] = train_data.processed_tweet.apply(sb_stem)


In [31]:
train_data

,tweetText,polarity_value,processed_tweet,stemmed_tweet
0,@PauladeLasHeras No te libraras de ayudar me/n...,0,No te libraras de ayudar me nos Besos y gracias,"[libr, ayud, nos, bes, graci]"
1,@marodriguezb Gracias MAR,1,Gracias MAR,"[graci, mar]"
2,"Off pensando en el regalito Sinde, la que se v...",-1,Off pensando en el regalito Sinde la que se va...,"[off, pens, regalit, sind, que, sga, cuando, v..."
3,Conozco a alguien q es adicto al drama! Ja ja ...,1,Conozco a alguien q es adicto al drama Ja ja ...,"[conozc, algui, adict, dram, suen, algo]"
4,Toca @crackoviadeTV3 . Grabación dl especial N...,1,Toca Grabación dl especial Navideño Mari cri...,"[toc, grabacion, especial, navideñ, mari, crism]"
...,...,...,...,...
5731,Muy indignante si ...nadie repara en ello hoy ...,-1,Muy indignante si nadie repara en ello hoy aq...,"[muy, indign, si, nadi, rep, ello, hoy, aqui, ..."
5732,Más pobres por discriminar a la mujer http://t...,-1,Más pobres por discriminar a la mujer,"[más, pobr, por, discrimin, muj]"
5733,Crean un banco de productos para mujeres con c...,1,Crean un banco de productos para mujeres con c...,"[cre, banc, product, para, mujer, con, canc, mam]"
5734,Sobre la sorprendente huída hoy en el Senado d...,-1,Sobre la sorprendente huída hoy en el Senado d...,"[sobre, sorprendent, huid, hoy, sen, rajoy]"


In [32]:
amlo_tweets

,tweetText,processed_tweet,stemmed_tweet
0,"Sí, también misóginos y arribistas.",Sí también misóginos y arribistas,"[también, misogin, arrib]"
1,No puedo entender como “hombres” que se presum...,No puedo entender como hombres que se presumen...,"[pued, entend, como, hombr, que, presum, intel..."
2,Comunista de ultra derecha,Comunista de ultra derecha,"[comun, ultra, derech]"
3,Gran presidente apoyo total a nuestro presiden...,Gran presidente apoyo total a nuestro presiden...,"[gran, president, apoy, total, nuestro, presid..."
4,"Con la panzota que se carga ??? No mame , no e...",Con la panzota que se carga No mame no es cre...,"[con, panzot, que, carg, mam, no, creibl, que,..."
...,...,...,...
1055,@lopezobrador_ Y vemos en tv cualquier cantida...,Y vemos en tv cualquier cantidad de mujeres e...,"[vem, cualqui, cantid, mujer, extranjer, traid..."
1056,@lopezobrador_ La trata de personas vemos un ...,La trata de personas vemos un desfile de muje...,"[trat, person, vem, desfil, mujer, vest, muy, ..."
1057,@lopezobrador_ debe estar muy orgulloso por ha...,debe estar muy orgulloso por haber pasado ya ...,"[deb, estar, muy, orgull, por, hab, pas, histo..."
1058,@lopezobrador_ anunciará el 15 de Septiembre q...,anunciará el 15 de Septiembre que se suma a l...,"[anunci, septiembr, que, sum, movimient, antor..."


### N-Grams

In [33]:
from nltk import ngrams

def bigramize(tweets, n=2):
    bigrams=[]
    for tweet in tweets:
        bigrams += ngrams(tweets,n=2)
    return bigrams

def trigramize(tweets, n=3):
    trigrams=[]
    for tweet in tweets:
        trigrams += ngrams(tweets,n=3)
    return trigrams

In [34]:
train_data['bigrams'] = train_data.stemmed_tweet.apply(bigramize)
train_data['trigrams'] = train_data.stemmed_tweet.apply(trigramize)

<ipython-input-34-e8ddb862bb57>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['bigrams'] = train_data.stemmed_tweet.apply(bigramize)
<ipython-input-34-e8ddb862bb57>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['trigrams'] = train_data.stemmed_tweet.apply(trigramize)


In [35]:
train_data

,tweetText,polarity_value,processed_tweet,stemmed_tweet,bigrams,trigrams
0,@PauladeLasHeras No te libraras de ayudar me/n...,0,No te libraras de ayudar me nos Besos y gracias,"[libr, ayud, nos, bes, graci]","[(libr, ayud), (ayud, nos), (nos, bes), (bes, ...","[(libr, ayud, nos), (ayud, nos, bes), (nos, be..."
1,@marodriguezb Gracias MAR,1,Gracias MAR,"[graci, mar]","[(graci, mar), (graci, mar)]",[]
2,"Off pensando en el regalito Sinde, la que se v...",-1,Off pensando en el regalito Sinde la que se va...,"[off, pens, regalit, sind, que, sga, cuando, v...","[(off, pens), (pens, regalit), (regalit, sind)...","[(off, pens, regalit), (pens, regalit, sind), ..."
3,Conozco a alguien q es adicto al drama! Ja ja ...,1,Conozco a alguien q es adicto al drama Ja ja ...,"[conozc, algui, adict, dram, suen, algo]","[(conozc, algui), (algui, adict), (adict, dram...","[(conozc, algui, adict), (algui, adict, dram),..."
4,Toca @crackoviadeTV3 . Grabación dl especial N...,1,Toca Grabación dl especial Navideño Mari cri...,"[toc, grabacion, especial, navideñ, mari, crism]","[(toc, grabacion), (grabacion, especial), (esp...","[(toc, grabacion, especial), (grabacion, espec..."
...,...,...,...,...,...,...
5731,Muy indignante si ...nadie repara en ello hoy ...,-1,Muy indignante si nadie repara en ello hoy aq...,"[muy, indign, si, nadi, rep, ello, hoy, aqui, ...","[(muy, indign), (indign, si), (si, nadi), (nad...","[(muy, indign, si), (indign, si, nadi), (si, n..."
5732,Más pobres por discriminar a la mujer http://t...,-1,Más pobres por discriminar a la mujer,"[más, pobr, por, discrimin, muj]","[(más, pobr), (pobr, por), (por, discrimin), (...","[(más, pobr, por), (pobr, por, discrimin), (po..."
5733,Crean un banco de productos para mujeres con c...,1,Crean un banco de productos para mujeres con c...,"[cre, banc, product, para, mujer, con, canc, mam]","[(cre, banc), (banc, product), (product, para)...","[(cre, banc, product), (banc, product, para), ..."
5734,Sobre la sorprendente huída hoy en el Senado d...,-1,Sobre la sorprendente huída hoy en el Senado d...,"[sobre, sorprendent, huid, hoy, sen, rajoy]","[(sobre, sorprendent), (sorprendent, huid), (h...","[(sobre, sorprendent, huid), (sorprendent, hui..."


In [36]:
amlo_tweets['bigrams'] = amlo_tweets.stemmed_tweet.apply(bigramize)
amlo_tweets['trigrams'] = amlo_tweets.stemmed_tweet.apply(trigramize)

In [37]:
amlo_tweets

,tweetText,processed_tweet,stemmed_tweet,bigrams,trigrams
0,"Sí, también misóginos y arribistas.",Sí también misóginos y arribistas,"[también, misogin, arrib]","[(también, misogin), (misogin, arrib), (tambié...","[(también, misogin, arrib), (también, misogin,..."
1,No puedo entender como “hombres” que se presum...,No puedo entender como hombres que se presumen...,"[pued, entend, como, hombr, que, presum, intel...","[(pued, entend), (entend, como), (como, hombr)...","[(pued, entend, como), (entend, como, hombr), ..."
2,Comunista de ultra derecha,Comunista de ultra derecha,"[comun, ultra, derech]","[(comun, ultra), (ultra, derech), (comun, ultr...","[(comun, ultra, derech), (comun, ultra, derech..."
3,Gran presidente apoyo total a nuestro presiden...,Gran presidente apoyo total a nuestro presiden...,"[gran, president, apoy, total, nuestro, presid...","[(gran, president), (president, apoy), (apoy, ...","[(gran, president, apoy), (president, apoy, to..."
4,"Con la panzota que se carga ??? No mame , no e...",Con la panzota que se carga No mame no es cre...,"[con, panzot, que, carg, mam, no, creibl, que,...","[(con, panzot), (panzot, que), (que, carg), (c...","[(con, panzot, que), (panzot, que, carg), (que..."
...,...,...,...,...,...
1055,@lopezobrador_ Y vemos en tv cualquier cantida...,Y vemos en tv cualquier cantidad de mujeres e...,"[vem, cualqui, cantid, mujer, extranjer, traid...","[(vem, cualqui), (cualqui, cantid), (cantid, m...","[(vem, cualqui, cantid), (cualqui, cantid, muj..."
1056,@lopezobrador_ La trata de personas vemos un ...,La trata de personas vemos un desfile de muje...,"[trat, person, vem, desfil, mujer, vest, muy, ...","[(trat, person), (person, vem), (vem, desfil),...","[(trat, person, vem), (person, vem, desfil), (..."
1057,@lopezobrador_ debe estar muy orgulloso por ha...,debe estar muy orgulloso por haber pasado ya ...,"[deb, estar, muy, orgull, por, hab, pas, histo...","[(deb, estar), (estar, muy), (muy, orgull), (o...","[(deb, estar, muy), (estar, muy, orgull), (muy..."
1058,@lopezobrador_ anunciará el 15 de Septiembre q...,anunciará el 15 de Septiembre que se suma a l...,"[anunci, septiembr, que, sum, movimient, antor...","[(anunci, septiembr), (septiembr, que), (que, ...","[(anunci, septiembr, que), (septiembr, que, su..."


### Vectorization

In [38]:
train_data2 = train_data[['polarity_value','stemmed_tweet','bigrams','trigrams']]
train_data2

,polarity_value,stemmed_tweet,bigrams,trigrams
0,0,"[libr, ayud, nos, bes, graci]","[(libr, ayud), (ayud, nos), (nos, bes), (bes, ...","[(libr, ayud, nos), (ayud, nos, bes), (nos, be..."
1,1,"[graci, mar]","[(graci, mar), (graci, mar)]",[]
2,-1,"[off, pens, regalit, sind, que, sga, cuando, v...","[(off, pens), (pens, regalit), (regalit, sind)...","[(off, pens, regalit), (pens, regalit, sind), ..."
3,1,"[conozc, algui, adict, dram, suen, algo]","[(conozc, algui), (algui, adict), (adict, dram...","[(conozc, algui, adict), (algui, adict, dram),..."
4,1,"[toc, grabacion, especial, navideñ, mari, crism]","[(toc, grabacion), (grabacion, especial), (esp...","[(toc, grabacion, especial), (grabacion, espec..."
...,...,...,...,...
5731,-1,"[muy, indign, si, nadi, rep, ello, hoy, aqui, ...","[(muy, indign), (indign, si), (si, nadi), (nad...","[(muy, indign, si), (indign, si, nadi), (si, n..."
5732,-1,"[más, pobr, por, discrimin, muj]","[(más, pobr), (pobr, por), (por, discrimin), (...","[(más, pobr, por), (pobr, por, discrimin), (po..."
5733,1,"[cre, banc, product, para, mujer, con, canc, mam]","[(cre, banc), (banc, product), (product, para)...","[(cre, banc, product), (banc, product, para), ..."
5734,-1,"[sobre, sorprendent, huid, hoy, sen, rajoy]","[(sobre, sorprendent), (sorprendent, huid), (h...","[(sobre, sorprendent, huid), (sorprendent, hui..."


In [93]:
train_bigrams = train_data2['bigrams'].value_counts()
train_bigrams

[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [94]:
from collections import Counter
Counter = Counter(train_bigrams)
most_occur = Counter.most_common(4)
most_occur

[(1, 5629), (2, 6), (10, 2), (9, 2)]

In [95]:
from itertools import groupby
[len(list(group)) for key, group in groupby(train_bigrams)]

[1, 2, 2, 1, 2, 6, 5629]

In [59]:
train_bigrams_top = train_bigrams[0:4]
train_bigrams_top

[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  40
[(notici, descombac), (descombac, está), (está, dispon), (notici, descombac), (descombac, está), (está, dispon), 

In [42]:
train_trigrams = train_data2['trigrams'].value_counts()
train_trigrams

[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [43]:
train_trigrams_top = train_trigrams[0:4]
train_trigrams_top

[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [44]:
train_data3 = train_data2.copy()
for index, row in train_data2.iterrows():
    for word in row.stemmed_tweet:
        if word not in train_data3.columns:
            train_data3[word] = 0
        train_data3.loc[index, word] = 1
train_data3

,polarity_value,stemmed_tweet,bigrams,trigrams,libr,ayud,nos,bes,graci,mar,...,ambas,dejacion,ferraris,buzon,ubic,uhy4gmfn,rick,santorum,jiraf,huid
0,0,"[libr, ayud, nos, bes, graci]","[(libr, ayud), (ayud, nos), (nos, bes), (bes, ...","[(libr, ayud, nos), (ayud, nos, bes), (nos, be...",1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,"[graci, mar]","[(graci, mar), (graci, mar)]",[],0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,-1,"[off, pens, regalit, sind, que, sga, cuando, v...","[(off, pens), (pens, regalit), (regalit, sind)...","[(off, pens, regalit), (pens, regalit, sind), ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,"[conozc, algui, adict, dram, suen, algo]","[(conozc, algui), (algui, adict), (adict, dram...","[(conozc, algui, adict), (algui, adict, dram),...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,"[toc, grabacion, especial, navideñ, mari, crism]","[(toc, grabacion), (grabacion, especial), (esp...","[(toc, grabacion, especial), (grabacion, espec...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5731,-1,"[muy, indign, si, nadi, rep, ello, hoy, aqui, ...","[(muy, indign), (indign, si), (si, nadi), (nad...","[(muy, indign, si), (indign, si, nadi), (si, n...",0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5732,-1,"[más, pobr, por, discrimin, muj]","[(más, pobr), (pobr, por), (por, discrimin), (...","[(más, pobr, por), (pobr, por, discrimin), (po...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5733,1,"[cre, banc, product, para, mujer, con, canc, mam]","[(cre, banc), (banc, product), (product, para)...","[(cre, banc, product), (banc, product, para), ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5734,-1,"[sobre, sorprendent, huid, hoy, sen, rajoy]","[(sobre, sorprendent), (sorprendent, huid), (h...","[(sobre, sorprendent, huid), (sorprendent, hui...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [56]:
#train_data_grams = pd.concat([train_data3, pd.DataFrame(columns=train_bigrams_top)])
#train_data_grams

train_data_grams = train_data3.copy()

In [ ]:
#train_data_grams = pd.concat([train_data_grams, pd.DataFrame(columns=train_trigrams_top)])
#train_data_grams

In [65]:
train_data_grams['(notici, descombac)']=0
train_data_grams['(descombac, está)']=0
train_data_grams['(está, dispon)']=0
train_data_grams['(muy, buen)']=0
train_data_grams['(buen, noch)']=0
train_data_grams['(mejor, timelin)']=0
train_data_grams['(timelin, está)']=0
train_data_grams['(está, dispon)']=0
train_data_grams['(dispon, histori)']=0
train_data_grams['(histori, del)']=0
train_data_grams['(dia, por)']=0

train_data_grams['(notici, descombac, está)']=0
train_data_grams['(descombac, está, dispon)']=0
train_data_grams['(muy, buen, noch)']=0
train_data_grams['(buen, noch, followercet)']=0
train_data_grams['(está, dispon, histori)']=0
train_data_grams['(dispon, histori, del)']=0
train_data_grams['(histori, del, dia)']=0

train_data_grams

,polarity_value,stemmed_tweet,bigrams,trigrams,libr,ayud,nos,bes,graci,mar,...,"(dispon, histori)","(histori, del)","(dia, por)","(notici, descombac, está)","(descombac, está, dispon)","(muy, buen, noch)","(buen, noch, followercet)","(está, dispon, histori)","(dispon, histori, del)","(histori, del, dia)"
0,0,"[libr, ayud, nos, bes, graci]","[(libr, ayud), (ayud, nos), (nos, bes), (bes, ...","[(libr, ayud, nos), (ayud, nos, bes), (nos, be...",1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,"[graci, mar]","[(graci, mar), (graci, mar)]",[],0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,-1,"[off, pens, regalit, sind, que, sga, cuando, v...","[(off, pens), (pens, regalit), (regalit, sind)...","[(off, pens, regalit), (pens, regalit, sind), ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,"[conozc, algui, adict, dram, suen, algo]","[(conozc, algui), (algui, adict), (adict, dram...","[(conozc, algui, adict), (algui, adict, dram),...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,"[toc, grabacion, especial, navideñ, mari, crism]","[(toc, grabacion), (grabacion, especial), (esp...","[(toc, grabacion, especial), (grabacion, espec...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5731,-1,"[muy, indign, si, nadi, rep, ello, hoy, aqui, ...","[(muy, indign), (indign, si), (si, nadi), (nad...","[(muy, indign, si), (indign, si, nadi), (si, n...",0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5732,-1,"[más, pobr, por, discrimin, muj]","[(más, pobr), (pobr, por), (por, discrimin), (...","[(más, pobr, por), (pobr, por, discrimin), (po...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5733,1,"[cre, banc, product, para, mujer, con, canc, mam]","[(cre, banc), (banc, product), (product, para)...","[(cre, banc, product), (banc, product, para), ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5734,-1,"[sobre, sorprendent, huid, hoy, sen, rajoy]","[(sobre, sorprendent), (sorprendent, huid), (h...","[(sobre, sorprendent, huid), (sorprendent, hui...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
train_data4 = train_data_grams.copy()
for index, row in train_data4.iterrows():
    for word in row.bigrams:
        train_data4.loc[index, word] = 1
train_data4

KeyboardInterrupt: 

In [ ]:
train_data5 = train_data4.copy()
for index, row in train_data5.iterrows():
    for word in row.trigrams:
        train_data5.loc[index, word] = 1
train_data5

In [ ]:
train_data_final = train_data5.replace(np.NaN, 0)
train_data_final

In [ ]:
train_data_final.to_csv('train_data.csv')

In [ ]:
amlo_tweets

In [ ]:
amlo_tweets2 = amlo_tweets[['stemmed_tweet','bigrams','trigrams']]

In [ ]:
amlo_bigrams = amlo_tweets2['bigrams'].value_counts()
amlo_bigrams

In [ ]:
amlo_bigrams_top = amlo_bigrams[0:4]
amlo_bigrams_top

In [ ]:
amlo_trigrams = amlo_tweets2['trigrams'].value_counts()
amlo_trigrams

In [ ]:
amlo_trigrams_top = amlo_trigrams[0:4]
amlo_trigrams_top

In [ ]:
amlo_tweets3 = amlo_tweets2.copy()
for index, row in amlo_tweets2.iterrows():
    for word in row.stemmed_tweet:
        if word not in amlo_tweets3.columns:
            amlo_tweets3[word] = 0
        amlo_tweets3.loc[index, word] = 1
amlo_tweets3

In [ ]:
amlo_tweets_grams = pd.concat([amlo_tweets3, pd.DataFrame(columns=amlo_bigrams_top)])
amlo_tweets_grams

In [ ]:
amlo_tweets_grams = pd.concat([amlo_tweets_grams, pd.DataFrame(columns=amlo_trigrams_top)])
amlo_tweets_grams

In [ ]:
amlo_tweets4 = amlo_tweets_grams.copy()
for index, row in amlo_tweets4.iterrows():
    for word in row.bigrams:
        amlo_tweets4.loc[index, word] = 1
amlo_tweets4

In [ ]:
amlo_tweets5 = amlo_tweets4.copy()
for index, row in amlo_tweets5.iterrows():
    for word in row.trigrams:
        amlo_tweets5.loc[index, word] = 1
amlo_tweets5

In [ ]:
amlo_tweets_final = amlo_tweets5.replace(np.NaN, 0)
amlo_tweets_final

In [ ]:
amlo_tweets_final.to_csv('amlo_tweets.csv')

# Classification with Naive Bayes

In [ ]:
X = train_data_final.drop(['polarity_value','stemmed_tweet','bigrams','trigrams'], axis=1)
X

In [ ]:
y = train_data_final[['polarity_value']]
y

In [ ]:
X_test = amlo_tweets_final.drop(['stemmed_tweet','bigrams','trigrams'], axis=1)
X_test

In [ ]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42) # 70% training and 30% test

In [ ]:
#Create a Gaussian Classifier
nb = GaussianNB()

#Train the model using the training sets
nb.fit(X_train, y_train)

In [ ]:
#Predict the response for test dataset
y_pred = nb.predict(X_test)
y_pred

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)  

print(cm)  

## Frequently used words

In [ ]:
import string
punctuation = list(string.punctuation)
stop = stopwords.words('spanish') + punctuation + ['rt', 'via'] + ['lopezobrador'] 
wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stop,
                          max_words=200,
                          max_font_size=50, 
                          random_state=42
                         ).generate(str(amlo_tweets['processed_tweet']))
plt.imshow(wordcloud)
plt.rcParams["figure.figsize"] = (15,15)
plt.axis('off')
plt.show()